Importing Libraries

In [1]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

Mediapipe Holistic

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
def detection(image,model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable=False
    results = model.process(image)
    image.flags.writeable=False
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image,results


In [4]:
def draw(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
        
        # 2. Right hand
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))

        # 3. Left Hand
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))

        # 4. Pose Detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [17]:
# capture=cv.VideoCapture(0)
# capture.set(3, 640)
# capture.set(4, 480)
# capture.set(10, 70)  
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while capture.isOpened():
#         ret, frame = capture.read()

#         image,results = detection(frame,holistic);

#         draw(image,results)

#         cv.imshow('Video', image)

#         if cv.waitKey(10) & 0xFF == ord('q'):
#             break

#     capture.release()
#     cv.destroyAllWindows()

Extract Values

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

Collect values

In [6]:
d_path = os.path.join('MP_data')
actions=np.array(['Hello','ThankYou','Goodbye'])
no_sequences=30  #thirty videos worth of data
sequence_length= 30  #videos are going to be 30 frames in length


for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(d_path,action,str(sequence)))
        except:
            pass

Collect keypoints for training and testing

In [7]:
capture=cv.VideoCapture(0)
capture.set(3, 640)
capture.set(4, 480)
capture.set(10, 70)  
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = capture.read()

                image,results = detection(frame,holistic);

                draw(image,results)

                #IMPORTANT
                if frame_num == 0:
                    cv.putText(image,'STARTING COLLECTION',(120,200),cv.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv.LINE_AA)
                    cv.putText(image,'Collecting frames for {} video number {}'.format(action,sequence),(15,12),cv.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv.LINE_AA)
                    cv.waitKey(2000)
                else:
                    cv.putText(image,'Collecting frames for {} video number {}'.format(action,sequence),(15,12),cv.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv.LINE_AA)

                keypoints=extract_keypoints(results)
                npy_path = os.path.join(d_path,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)

                cv.imshow('Video', image)

                if cv.waitKey(10) & 0xFF == ord('q'):
                    break

    capture.release()
    cv.destroyAllWindows()

Preprocess data

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}  #labelled all actions

sequences,labels=[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(d_path,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [10]:
np.array(sequences).shape

(90, 30, 1662)

In [11]:
np.array(labels).shape

(90,)

In [12]:
X=np.array(sequences)
y=to_categorical(labels).astype(int)   #one hot encoding of labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 1)

LSTM Neural Network

In [40]:
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.callbacks import TensorBoard

In [41]:
log_dir = os.path.join('Logs')
callback = TensorBoard(log_dir=log_dir)    #tensorboard helps monitor neural network progress

In [44]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(LSTM(64,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

model.summary()


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_42 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_43 (LSTM)              (None, 30, 64)            33024     
                                                                 
 lstm_44 (LSTM)              (None, 64)                33024     
                                                                 
 dense_42 (Dense)            (None, 64)                4160      
                                                                 
 dense_43 (Dense)            (None, 32)                2080      
                                                                 
 dense_44 (Dense)            (None, 3)                 99        
                                                                 
Total params: 514,499
Trainable params: 514,499
Non-t

In [45]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

model.fit(X_train,y_train,epochs=2000,callbacks=[callback])

Epoch 1/2000
3/3 [==============================] - 4s 98ms/step - loss: 25.2952 - categorical_accuracy: 0.2824
Epoch 2/2000
3/3 [==============================] - 0s 92ms/step - loss: 18.5678 - categorical_accuracy: 0.4000
Epoch 3/2000
3/3 [==============================] - 0s 80ms/step - loss: 23.8146 - categorical_accuracy: 0.2588
Epoch 4/2000
3/3 [==============================] - 0s 80ms/step - loss: 23.5446 - categorical_accuracy: 0.4235
Epoch 5/2000
3/3 [==============================] - 0s 77ms/step - loss: 24.7486 - categorical_accuracy: 0.3176
Epoch 6/2000
3/3 [==============================] - 0s 85ms/step - loss: 52.0569 - categorical_accuracy: 0.3294
Epoch 7/2000
3/3 [==============================] - 0s 80ms/step - loss: 7.4358 - categorical_accuracy: 0.4000
Epoch 8/2000
3/3 [==============================] - 0s 83ms/step - loss: 5.9013 - categorical_accuracy: 0.2941
Epoch 9/2000
3/3 [==============================] - 0s 91ms/step - loss: 7.5059 - categorical_accuracy: 0.

KeyboardInterrupt: 

In [46]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_42 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_43 (LSTM)              (None, 30, 64)            33024     
                                                                 
 lstm_44 (LSTM)              (None, 64)                33024     
                                                                 
 dense_42 (Dense)            (None, 64)                4160      
                                                                 
 dense_43 (Dense)            (None, 32)                2080      
                                                                 
 dense_44 (Dense)            (None, 3)                 99        
                                                                 
Total params: 514,499
Trainable params: 514,499
Non-t

Make predictions

In [52]:
res=model.predict(X_test)
actions[np.argmax(res[4])]


1/1 [==============================] - 0s 41ms/step


'Goodbye'

In [53]:
actions[np.argmax(y_test[4])]

'Goodbye'

Save model

In [54]:
model.save('action.h5')
#del model
#model.load_weights('action.h5)

Evaluation

In [55]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat = model.predict(X_train)
ytrue=np.argmax(y_train,axis=1).tolist()
yhat=np.argmax(yhat,axis=1).tolist()

multilabel_confusion_matrix(ytrue,yhat)
accuracy_score(ytrue,yhat)

3/3 [==============================] - 0s 36ms/step


0.8941176470588236

Testing and trying the model

In [60]:
from scipy import stats
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv.putText(output_frame, actions[num], (0, 85+num*40), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
        
    return output_frame

In [63]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv.putText(image, ' '.join(sentence), (3,30), 
                       cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
        
        # Show to screen
        cv.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti